### 4d_summarize_results.ipynb  
Passes the validated chunks back to LLaMA for final summarization.  
Formats a clean final answer for the user along with context references.

In [1]:
from langchain.chat_models import ChatOllama
from langchain.schema import SystemMessage, HumanMessage
from langchain.vectorstores import FAISS
from langchain.embeddings import HuggingFaceEmbeddings

In [2]:
llm = ChatOllama(model="llama3")

/var/folders/90/p8kgtvm572d51cr90k19v_580000gn/T/ipykernel_8110/368174091.py:1: LangChainDeprecationWarning: The class `ChatOllama` was deprecated in LangChain 0.3.1 and will be removed in 1.0.0. An updated version of the class exists in the :class:`~langchain-ollama package and should be used instead. To use it run `pip install -U :class:`~langchain-ollama` and import as `from :class:`~langchain_ollama import ChatOllama``.
  llm = ChatOllama(model="llama3")


create embedder object + load vectordb

In [3]:
embedder = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")
vectorstore = FAISS.load_local("vectorstore/", embeddings=embedder, allow_dangerous_deserialization=True)

/var/folders/90/p8kgtvm572d51cr90k19v_580000gn/T/ipykernel_8110/2066271422.py:1: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embedder = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")
/opt/anaconda3/envs/ig-reel-env/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
query = "battery duration of hip and ankle configurations"

same process (this needs to be replaced and held in state when doing langgraph)

In [5]:
matches = vectorstore.similarity_search(query, k=3)

In [6]:
retrieved_text = "\n\n".join([f"[chunk {i+1}]\n{doc.page_content}" for i, doc in enumerate(matches)])

craft rlly good prompt LOL

In [7]:
messages = [
    SystemMessage(
        content=(
            "You are a helpful assistant. Given a user query and some context chunks, "
            "summarize the answer clearly and concisely. If the context is unclear, say so."
        )
    ),
    HumanMessage(
        content=(
            f"User query: {query}\n\n"
            f"Context chunks:\n{retrieved_text}\n\n"
            "Please provide a concise answer. You may mention which chunk(s) supported your response."
        )
    )
]

In [8]:
response = llm.invoke(messages)

In [10]:
print(response.content)

According to the provided context, the battery duration of hip-and-ankle configurations is 25 minutes before reaching 50% of the voltage capacity (~900 mAh). This information can be found in Chunk 1.

Additionally, when configured for simultaneous assistance to both hip and ankle joints, the device was able to operate for approximately 35 minutes (Chunk 3).
